
<img src="../../docs/images/DSPy8.png" alt="DSPy7 Image" height="150"/>

## **DSPy断言**: 对基础模型施加计算约束

### **TweetGen**: 生成推文以回答问题

[<img align="center" src="https://colab.research.google.com/assets/colab-badge.svg" />](https://colab.research.google.com/github/stanfordnlp/dspy/blob/main/examples/tweets/tweets_assertions.ipynb)


这个笔记本突出了[**DSPy Assertions**](https://dspy-docs.vercel.app/docs/building-blocks/assertions)的一个示例，允许在DSPy程序中声明计算约束。


这个笔记本建立在**DSPy**框架的基本概念之上。在阅读本笔记本之前，需要先完成[DSPy教程](../../intro.ipynb)、[**DSPy Assertions文档**](https://dspy-docs.vercel.app/docs/building-blocks/assertions)以及LongFormQA中的DSPy Assertions入门教程。

In [ ]:
# 克隆指定的git仓库
!git clone https://huggingface.co/arnavs11/DSPy_TweetGen_Cache
%cd DSPy_TweetGen_Cache/
# 切换到master分支
!git checkout master
%cd ..
import os
repo_clone_path = '/content/DSPy_TweetGen_Cache'

# 检查'/content'目录是否可写
if not os.access('/content', os.W_OK):
    # 如果'/content'目录不可写，选择一个替代目录
    # 例如：使用相对于当前工作目录的目录
    repo_clone_path = os.path.join(os.getcwd(), 'DSPy_TweetGen_Cache')

# 为此笔记本设置缓存
os.environ["DSP_NOTEBOOK_CACHEDIR"] = repo_clone_path

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os
import regex as re
import json

try: # 当在谷歌Colab上时，让我们克隆笔记本以便下载缓存。
    import google.colab
    repo_path = 'dspy'
    
    !git -C $repo_path pull origin || git clone https://github.com/stanfordnlp/dspy $repo_path
except:
    repo_path = '.'

if repo_path not in sys.path:
    sys.path.append(repo_path)


import pkg_resources # 如果尚未安装该软件包，请安装该软件包
if not "dspy-ai" in {pkg.key for pkg in pkg_resources.working_set}:
    !pip install -U pip
    !pip install dspy-ai
    !pip install openai~=0.28.1
    !pip install -e $repo_path

import dspy
from dspy.predict import Retry
from dspy.datasets import HotPotQA
from dspy.teleprompt import BootstrapFewShotWithRandomSearch
from dsp.utils import deduplicate
from dspy.evaluate.evaluate import Evaluate
from dspy.primitives.assertions import assert_transform_module, backtrack_handler

In [ ]:
# 创建一个 ColBERTv2 模型，使用给定的 URL 连接到 wiki17_abstracts 数据集
colbertv2_wiki17_abstracts = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')

# 配置默认设置，将 ColBERTv2 模型设置为默认的检索模型
dspy.settings.configure(rm=colbertv2_wiki17_abstracts)

# 创建一个 OpenAI 模型，使用 'gpt-3.5-turbo-0613' 模型和最大 token 数为 500
turbo = dspy.OpenAI(model='gpt-3.5-turbo-0613', max_tokens=500)

# 配置默认设置，将 OpenAI 模型设置为默认的语言模型，同时设置 trace 为空列表，温度为 0.7
dspy.settings.configure(lm=turbo, trace=[], temperature=0.7)

In [ ]:
# 导入HotPotQA数据集
dataset = HotPotQA(train_seed=1, train_size=300, eval_seed=2023, dev_size=300, test_size=0, keep_details=True)

# 从训练集中提取问题和答案作为输入
trainset = [x.with_inputs('question', 'answer') for x in dataset.train]

# 从验证集中提取问题和答案作为输入
devset = [x.with_inputs('question', 'answer') for x in dataset.dev]

### 3] TweetGen

让我们介绍一个新任务：TweetGen。我们扩展了`Multi-Hop QA`程序，但现在的目标是以推文的形式呈现答案生成。

`Tweeter`模块捕获了从`Multi-Hop QA`中的查询生成、段落检索和上下文组装中的迭代多跳生成过程。`GenerateTweet`层现在利用上下文和问题一起生成一个有效回答问题的推文。

通过这个程序，我们的目标是生成符合以下准则的推文：
1. 推文没有标签。
2. 推文包含正确答案
3. 推文在字符限制内。
4. 推文具有吸引力
5. 推文忠实

In [ ]:
class GenerateSearchQuery(dspy.Signature):
    """编写一个简单的搜索查询，以帮助回答一个复杂的问题。"""
    context = dspy.InputField(desc="可能包含相关事实")
    question = dspy.InputField()
    query = dspy.OutputField()

class GenerateTweet(dspy.Signature):
    """生成一个引人入胜的推文，有效地回答一个问题，忠实于上下文，不超过280个字符，并且没有标签。"""
    question = dspy.InputField()
    context = dspy.InputField(desc="可能包含相关事实")
    tweet = dspy.OutputField()

class Tweeter(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_tweet = dspy.ChainOfThought(GenerateTweet)

    def forward(self, question, answer):
        context = []
        max_hops=2
        passages_per_hop=3
        generate_query = [dspy.ChainOfThought(GenerateSearchQuery) for _ in range(max_hops)]
        retrieve = dspy.Retrieve(k=passages_per_hop)
        for hop in range(max_hops):
            query = generate_query[hop](context=context, question=question).query
            passages = retrieve(query).passages
            context = deduplicate(context + passages)  # 去重
        generated_tweet = self.generate_tweet(question=question, context=context).tweet
        return dspy.Prediction(generated_tweet=generated_tweet, context=context)
    
tweeter = Tweeter()

### 4] 评估 - 内在和外在

#### 内在度量：通过内部计算约束是目标

**无标签** - 这是一个用户个性化的约束，用于测试模型能否遵循一个特定但简单的指导方针，即在生成的推文中不包含任何标签。

**正确答案包含** - 这是一个通用检查，以确保推文确实包含问题的正确答案。

**长度内** - 此检查遵循 Twitter 平台的指导方针，每条推文限制为 280 个字符。

**参与度** - 为了验证推文的参与质量，我们定义并调用另一个 **DSPy** 程序：``Predict`` 在 ``AssessTweet`` 上，依赖于相同的 LM 来回答问题：`“评估的文本是否构成一个独立的、引人入胜的推文？如果不引人入胜，请说不。”`

**忠实度** - 为了验证推文对其引用上下文的忠实度，我们同样使用上面的 `AssessTweet`，但用以下问题提示它：`“评估的文本是否基于上下文？如果包含不在上下文中的重要事实，请说不。”`

In [ ]:
import re
from daisy import dspy

def has_no_hashtags(text):
    # 检查文本中是否包含hashtags
    return len(re.findall(r"#\w+", text)) == 0

def is_within_length_limit(text, length_limit=280):
    # 检查文本长度是否在限制范围内
    return len(text) <= length_limit

def is_assessment_yes(assessment_answer):
    """检查评估答案的第一个单词是否为'yes'。"""
    return assessment_answer.split()[0].lower() == 'yes'

def has_correct_answer(text, answer):
    # 检查文本中是否包含正确答案
    return answer in text

class AssessTweet(dspy.Signature):
    """评估推文在指定维度上的质量。"""

    context = dspy.InputField(desc='如果不适用，请忽略')
    assessed_text = dspy.InputField()
    assessment_question = dspy.InputField()
    assessment_answer = dspy.OutputField(desc="是或否")

def no_hashtags_metric(gold, pred, trace=None):
    tweet = pred.generated_tweet
    no_hashtags = has_no_hashtags(tweet)
    score = no_hashtags
    return score

def is_correct_metric(gold, pred, trace=None):
    answer, tweet = gold.answer, pred.generated_tweet
    correct = has_correct_answer(tweet, answer)
    score = correct
    return score

def within_length_metric(gold, pred, trace=None):
    tweet = pred.generated_tweet
    within_length_limit = is_within_length_limit(tweet, 280)
    score = within_length_limit
    return score

def engaging_metric(gold, pred, trace=None):
    tweet = pred.generated_tweet
    engaging = "评估文本是否构成一个自包含、引人入胜的推文？如果不引人入胜，请说不。"
    engaging = dspy.Predict(AssessTweet)(context='N/A', assessed_text=tweet, assessment_question=engaging)
    engaging = engaging.assessment_answer.split()[0].lower() == 'yes'
    score = engaging
    return score

def faithful_metric(gold, pred, trace=None):
    context, tweet = pred.context, pred.generated_tweet
    faithful = "评估文本是否基于上下文？如果包含重要事实而上下文中没有，请说不。"   
    faithful = dspy.Predict(AssessTweet)(context=context, assessed_text=tweet, assessment_question=faithful)
    faithful = faithful.assessment_answer.split()[0].lower() == 'yes'
    score = faithful
    return score

#### 外部度量：评估生成输出在下游任务中的整体质量和有效性

外部度量被定义为生成推文在遵循所述约束条件方面的整体质量，并且这是通过一个综合度量来评估的。

在保持形成有效推文的最相关的内在度量（正确性和长度约束）的同时，整体综合度量返回5个内在度量的平均分数。

In [ ]:
def overall_metric(gold, pred, trace=None):
    # 提取gold和pred中的信息
    answer, context, tweet = gold.answer, pred.context, pred.generated_tweet
    # 检查tweet中是否有hashtags
    no_hashtags = has_no_hashtags(tweet)
    # 检查tweet是否在280个字符以内
    within_length_limit = is_within_length_limit(tweet, 280)
    # 检查tweet是否包含正确答案
    correct = has_correct_answer(tweet, answer)
    # 评估tweet是否构成一个自包含且引人入胜的推文
    engaging = "Does the assessed text make for a self-contained, engaging tweet? Say no if it is not engaging."
    # 评估tweet是否基于上下文
    faithful = "Is the assessed text grounded in the context? Say no if it includes significant facts not in the context."   
    # 使用dspy.Predict对tweet进行faithful和engaging的评估
    faithful = dspy.Predict(AssessTweet)(context=context, assessed_text=tweet, assessment_question=faithful)
    engaging = dspy.Predict(AssessTweet)(context='N/A', assessed_text=tweet, assessment_question=engaging)
    # 将评估结果转换为布尔值
    engaging, faithful = [m.assessment_answer.split()[0].lower() == 'yes' for m in [engaging, faithful]]
    # 计算得分
    score = (correct + engaging + faithful + no_hashtags + within_length_limit) if correct and within_length_limit else 0
    return score / 5.0

因此，我们将评估定义如下：

In [ ]:
# 定义评估指标列表
metrics = [no_hashtags_metric, is_correct_metric, within_length_metric, engaging_metric, faithful_metric, overall_metric]

# 遍历评估指标列表
for metric in metrics:
    # 创建 Evaluate 实例
    evaluate = Evaluate(metric=metric, devset=devset, num_threads=1, display_progress=True, display_table=5)
    # 对 tweeter 进行评估
    evaluate(tweeter)

让我们来看一个生成推文的示例：

In [ ]:
example = devset[118]  # 从devset中获取第118个样本
tweet = tweeter(question=example.question, answer=example.answer)  # 使用example的问题和答案创建一个tweet对象
print(f'生成的推文: ', tweet.generated_tweet)  # 打印生成的推文
tweet.context  # 输出tweet的上下文

In [ ]:
for metric in metrics:  # 遍历metrics列表中的每一个指标
    evaluate = Evaluate(metric=metric, devset=devset[118:119], num_threads=1, display_progress=True, display_table=5)  # 创建一个Evaluate对象，传入指标、数据集、线程数、显示进度和显示表格参数
    evaluate(tweeter)  # 对tweeter进行评估

在这个例子中，我们看到生成的推文长度为151个字符，位于280个字符的范围内。实际上，它包含了正确答案`Hooke`。

然而，它未能包含标签，我们在推文末尾看到了`#knowledge`。此外，这条推文被确定为缺乏吸引力，从视觉测试来看，它只是简单地陈述了答案而已。

让我们尝试修复这个问题，并使用DSPy断言生成推文。

### 5] 引入断言：带有断言的 TweeterWithAssertions

为了纠正这些不同的错误，让我们在 DSPy 断言语义中包含简单重申我们的计算约束的断言。

在第一个**断言**中，我们通过正则表达式检查生成的推文是否有任何标签，并且如果违反了，断言：**"请修改推文以删除其后的标签短语。"**

类似地，我们检查推文的长度，如果不在 280 个字符内，我们发送反馈信息：**"请确保推文在 {280} 个字符内。"**

我们检查生成的推文是否包含答案，如果没有，我们断言：**"推文不包括问题的正确答案。请相应修改。"**

对于参与度和忠实度检查，我们利用上面的设置，检查相应评估是否确定为`是`或`否`。

In [ ]:
class TweeterWithAssertions(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_tweet = dspy.ChainOfThought(GenerateTweet)

    def forward(self, question, answer):
        context = []
        max_hops = 2
        passages_per_hop = 3
        generate_query = [dspy.ChainOfThought(GenerateSearchQuery) for _ in range(max_hops)]
        retrieve = dspy.Retrieve(k=passages_per_hop)
        for hop in range(max_hops):
            # 生成查询语句
            query = generate_query[hop](context=context, question=question).query
            # 检索相关段落
            passages = retrieve(query).passages
            # 去重并更新上下文
            context = deduplicate(context + passages)
        # 生成推文
        generated_tweet = self.generate_tweet(question=question, context=context).tweet
        # 检查推文是否包含标签短语
        dspy.Suggest(has_no_hashtags(generated_tweet), f"请修改推文以删除后面的标签短语。", target_module=GenerateTweet)
        # 检查推文长度是否符合限制
        dspy.Suggest(is_within_length_limit(generated_tweet, 280), f"请确保推文在{280}个字符以内。", target_module=GenerateTweet)
        # 检查推文是否包含正确答案
        dspy.Suggest(has_correct_answer(generated_tweet, answer), "推文未包含正确答案。请相应修改。", target_module=GenerateTweet)
        engaging_question = "评估的文本是否构成一个自成一体、引人入胜的推文？如果不引人入胜，请选择否。"
        engaging_assessment = dspy.Predict(AssessTweet)(context=context, assessed_text=generated_tweet, assessment_question=engaging_question)
        # 检查推文是否引人入胜
        dspy.Suggest(is_assessment_yes(engaging_assessment.assessment_answer), "文本不够引人入胜。请修改以增加吸引力。", target_module=GenerateTweet)
        faithful_question = "评估的文本是否基于上下文？如果包含重要事实而上下文中没有，请选择否。"
        faithful_assessment = dspy.Predict(AssessTweet)(context='N/A', assessed_text=generated_tweet, assessment_question=faithful_question)
        # 检查推文是否基于上下文
        dspy.Suggest(is_assessment_yes(faithful_assessment.assessment_answer), "文本包含不忠实的元素或上下文中没有的重要事实。请修改以提高准确性。", target_module=GenerateTweet)
        return dspy.Prediction(generated_tweet=generated_tweet, context=context)

tweeter_with_assertions = assert_transform_module(TweeterWithAssertions().map_named_predictors(Retry), backtrack_handler)

让我们现在在开发集上评估`TweeterWithAssertions`。

In [ ]:
# 定义评估指标列表
metrics = [no_hashtags_metric, is_correct_metric, within_length_metric, engaging_metric, faithful_metric, overall_metric]

# 遍历每个评估指标
for metric in metrics:
    # 创建 Evaluate 实例，传入评估指标、开发集、线程数、显示进度和显示表格
    evaluate = Evaluate(metric=metric, devset=devset, num_threads=1, display_progress=True, display_table=5)
    # 对 tweeter_with_assertions 进行评估
    evaluate(tweeter_with_assertions)

现在让我们看一下，通过添加断言，我们生成的推文如何得到改进。

In [ ]:
example = devset[118]  # 从devset中获取第118个示例
tweet = tweeter_with_assertions(question=example.question, answer=example.answer)  # 使用示例的问题和答案创建一个tweet对象
print(f'生成的推文：', tweet.generated_tweet)  # 打印生成的推文
tweet.context  # 输出tweet的上下文

In [ ]:
# 对于每一个指标，创建一个Evaluate对象并进行评估
for metric in metrics:
    # 创建Evaluate对象，传入指标、开发集的子集、线程数、显示进度和显示表格
    evaluate = Evaluate(metric=metric, devset=devset[118:119], num_threads=1, display_progress=True, display_table=5)
    # 对tweeter_with_assertions进行评估
    evaluate(tweeter_with_assertions)

我们看到推文在遵循我们设定的所有限制条件后有了显著的改进！

它不再有标签，既引人入胜又忠实，同时在280个字符内包含了正确答案。令人兴奋！

### 6] 使用断言进行编译

我们可以利用 **DSPy** 的 `BootstrapFewShotWithRandomSearch` 优化器，自动生成少样本演示，并对候选进行随机搜索，输出最佳编译程序。我们通过 `overall_metric` 综合指标来评估这一过程。

我们可以首先在 `Tweeter` 上进行评估，看看在不包含断言的情况下编译的表现如何。

In [ ]:
# 创建一个BootstrapFewShotWithRandomSearch对象，使用overall_metric作为度量标准，最大bootstrapped示例数为2，候选程序数为6
teleprompter = BootstrapFewShotWithRandomSearch(metric=overall_metric, max_bootstrapped_demos=2, num_candidate_programs=6)
# 编译teleprompter，将学生模型tweeter作为学生和老师，使用trainset进行训练，使用devset的前100个示例进行验证
compiled_tweeter = teleprompter.compile(student=tweeter, teacher=tweeter, trainset=trainset, valset=devset[:100])

# 遍历metrics列表
for metric in metrics:
    # 创建一个Evaluate对象，使用指定的metric作为度量标准，devset作为验证集，使用1个线程，显示进度，每5个显示一次表格
    evaluate = Evaluate(metric=metric, devset=devset, num_threads=1, display_progress=True, display_table=5)
    # 对编译后的tweeter模型进行评估
    evaluate(compiled_tweeter)

现在我们在两种设置下进行带有断言的编译测试：

**带有断言的编译**：在编译过程中进行基于断言的示例引导和反例引导。老师拥有断言，而学生没有，学生从老师的基于断言的引导示例中学习。

**带有断言的编译 + 推理**：为老师和学生提供基于断言的优化，以在编译和推理过程中提供增强的基于断言的输出。

In [ ]:
# 创建一个BootstrapFewShotWithRandomSearch对象，使用overall_metric作为度量标准，最大引导演示次数为2，候选程序数量为6
teleprompter = BootstrapFewShotWithRandomSearch(metric=overall_metric, max_bootstrapped_demos=2, num_candidate_programs=6)
# 编译teleprompter对象，使用tweeter作为学生模型，tweeter_with_assertions作为教师模型，trainset作为训练集，devset的前100个样本作为验证集
compiled_with_assertions_tweeter = teleprompter.compile(student=tweeter, teacher=tweeter_with_assertions, trainset=trainset, valset=devset[:100])

# 遍历metrics列表中的每个度量标准
for metric in metrics:
    # 创建一个Evaluate对象，使用当前遍历到的metric作为度量标准，devset作为验证集，使用1个线程，显示进度，每次显示5个结果
    evaluate = Evaluate(metric=metric, devset=devset, num_threads=1, display_progress=True, display_table=5)
    # 对compiled_with_assertions_tweeter对象进行评估
    evaluate(compiled_with_assertions_tweeter)

In [ ]:
# 创建一个BootstrapFewShotWithRandomSearch对象，指定metric为overall_metric，最大bootstrapped示例数为2，候选程序数为6，线程数为1
teleprompter = BootstrapFewShotWithRandomSearch(metric=overall_metric, max_bootstrapped_demos=2, num_candidate_programs=6, num_threads=1)
# 编译tweeter_with_assertions模型，作为学生和教师模型，使用trainset进行训练，使用devset的前100个样本进行验证
compiled_tweeter_with_assertions = teleprompter.compile(student=tweeter_with_assertions, teacher=tweeter_with_assertions, trainset=trainset, valset=devset[:100])

# 遍历metrics列表
for metric in metrics:
    # 创建一个Evaluate对象，指定metric为当前遍历到的metric，devset为devset，线程数为1，显示进度，显示前5个结果
    evaluate = Evaluate(metric=metric, devset=devset, num_threads=1, display_progress=True, display_table=5)
    # 对编译后的tweeter_with_assertions模型进行评估
    evaluate(compiled_tweeter_with_assertions)